Importing Dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/stanford-dogs-dataset


Import Libraries

In [2]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Configuration And Preprocessing

In [3]:
# --- Configuration ---
DATASET_PATH = path
IMAGES_DIR = os.path.join(DATASET_PATH, 'images', 'Images')
ANNOTATIONS_DIR = os.path.join(DATASET_PATH, 'annotations','Annotation')
INPUT_SIZE = (224, 224)
NUM_CLASSES = 120

# Collect Image Paths and Annotations
image_paths = []
annotations = []
labels = []
label_map = {} # To map breed names to integer labels
next_label_id = 0

for breed_folder in os.listdir(IMAGES_DIR):
    breed_path = os.path.join(IMAGES_DIR, breed_folder)
    annotation_path_base = os.path.join(ANNOTATIONS_DIR, breed_folder)

    if os.path.isdir(breed_path):
        breed_name = breed_folder.split('-', 1)[1]
        if breed_name not in label_map:
            label_map[breed_name] = next_label_id
            next_label_id += 1
        class_id = label_map[breed_name]

        for img_name in os.listdir(breed_path):
            if img_name.endswith(('.jpg', '.jpeg', '.png')):
                img_file_path = os.path.join(breed_path, img_name)

                annotation_file_name = os.path.splitext(img_name)[0]
                annotation_file_path = os.path.join(annotation_path_base, annotation_file_name)

                if os.path.exists(annotation_file_path):
                    image_paths.append(img_file_path)
                    annotations.append(annotation_file_path)
                    labels.append(class_id)
                else:
                    print(f"Warning: Annotation not found for {img_file_path}")

print(f"Found {len(image_paths)} images with annotations.")
print(f"Found {len(label_map)} unique dog breeds.")

# Create inverse label map for decoding
id_to_breed = {v: k for k, v in label_map.items()}

# Custom Data Generator for Bounding Box Cropping
class DogDatasetGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def flow_from_data(self, image_paths, annotations, labels, target_size, batch_size=32, shuffle=True):
        self.image_paths = np.array(image_paths)
        self.annotations = np.array(annotations)
        self.labels = np.array(labels)
        self.target_size = target_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_samples = len(image_paths)

        if shuffle:
            indices = np.arange(self.num_samples)
            np.random.shuffle(indices)
            self.image_paths = self.image_paths[indices]
            self.annotations = self.annotations[indices]
            self.labels = self.labels[indices]

        i = 0
        while True:
            batch_images = []
            batch_labels = []

            for _ in range(self.batch_size):
                if i >= self.num_samples:
                    i = 0
                    if shuffle:
                        indices = np.arange(self.num_samples)
                        np.random.shuffle(indices)
                        self.image_paths = self.image_paths[indices]
                        self.annotations = self.annotations[indices]
                        self.labels = self.labels[indices]

                img_path = self.image_paths[i]
                anno_path = self.annotations[i]
                label = self.labels[i]

                try:
                    # Parse XML to get bounding box
                    tree = ET.parse(anno_path)
                    root = tree.getroot()
                    bndbox = root.find('object/bndbox')
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)

                    # Load and crop image
                    img = Image.open(img_path).convert('RGB')
                    cropped_img = img.crop((xmin, ymin, xmax, ymax))

                    # Resize and preprocess for ResNet
                    cropped_img = cropped_img.resize(self.target_size)
                    img_array = np.array(cropped_img)

                    # Apply Keras ResNet preprocessing
                    img_array = resnet_preprocess_input(img_array)

                    batch_images.append(img_array)
                    batch_labels.append(label)

                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    pass
                finally:
                    i += 1

            yield np.array(batch_images), np.array(batch_labels)

Found 20580 images with annotations.
Found 120 unique dog breeds.


Compile and Train Model

In [6]:
# Split Data
train_paths, test_paths, train_annos, test_annos, train_labels, test_labels = train_test_split(
    image_paths, annotations, labels, test_size=0.2, stratify=labels, random_state=42
)

train_paths, val_paths, train_annos, val_annos, train_labels, val_labels = train_test_split(
    train_paths, train_annos, train_labels, test_size=0.15, stratify=train_labels, random_state=42
)

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples: {len(test_paths)}")

# Instantiate Data Generators
train_datagen = DogDatasetGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = DogDatasetGenerator()
test_datagen = DogDatasetGenerator()

train_generator = train_datagen.flow_from_data(
    train_paths, train_annos, train_labels,
    target_size=INPUT_SIZE, batch_size=32, shuffle=True
)
val_generator = val_datagen.flow_from_data(
    val_paths, val_annos, val_labels,
    target_size=INPUT_SIZE, batch_size=32, shuffle=False
)
test_generator = test_datagen.flow_from_data(
    test_paths, test_annos, test_labels,
    target_size=INPUT_SIZE, batch_size=32, shuffle=False
)


# Load pre-trained ResNet model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE[0], INPUT_SIZE[1], 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the Model
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_dog_breed_model.keras', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_paths) // 32,
    epochs=50, # 25, 50, 100
    validation_data=val_generator,
    validation_steps=len(val_paths) // 32,
    callbacks=[early_stopping, model_checkpoint]
)

# Fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=len(train_paths) // 32,
    epochs=5,
    validation_data=val_generator,
    validation_steps=len(val_paths) // 32,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate on Test Set
model.load_weights('best_dog_breed_model.keras')
loss, accuracy = model.evaluate(test_generator, steps=len(test_paths) // 32)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Train samples: 13994
Validation samples: 2470
Test samples: 4116


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 24,675,448 (94.13 MB)

 Trainable params: 1,110,648 (4.24 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 24,675,448 (94.13 MB)

 Trainable params: 8,990,328 (34.30 MB)

 Non-trainable params: 15,685,120 (59.83 MB)

Epoch 1/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 108s 218ms/step - accuracy: 0.8709 - loss: 0.4674 - val_accuracy: 0.8182 - val_loss: 0.6296
Epoch 2/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 139s 320ms/step - accuracy: 0.9597 - loss: 0.1473 - val_accuracy: 0.8129 - val_loss: 0.6467
Epoch 3/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 142s 325ms/step - accuracy: 0.9890 - loss: 0.0591 - val_accuracy: 0.8170 - val_loss: 0.6769
Epoch 4/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 141s 324ms/step - accuracy: 0.9950 - loss: 0.0307 - val_accuracy: 0.8174 - val_loss: 0.6945
Epoch 5/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 90s 207ms/step - accuracy: 0.9971 - loss: 0.0213 - val_accuracy: 0.8105 - val_loss: 0.7703
128/128 ━━━━━━━━━━━━━━━━━━━━ 56s 437ms/step - accuracy: 0.8155 - loss: 0.6217
Test Loss: 0.5986
Test Accuracy: 0.8120


Save Models

In [7]:
model.save('dog_breed_classifier.keras')

Import Labels into JSON

In [8]:
import json

with open('id_to_breed.json', 'w') as f:
    json.dump(id_to_breed, f)